<a href="https://colab.research.google.com/github/afreskiw/Simple-Projects/blob/main/MSAccessConsolidator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pyodbc
import os
import glob

# Configuration
SOURCE_FOLDER = './data_sources'  # Folder containing your Access/Excel files
OUTPUT_FILE = 'consolidated_data.xlsx'

def get_access_data(filepath):
    """Connects to an Access DB and pulls data from all tables."""
    conn_str = (
        r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};'
        f'DBQ={os.path.abspath(filepath)};'
    )
    all_tables_data = {}
    try:
        with pyodbc.connect(conn_str) as conn:
            cursor = conn.cursor()
            # Get list of user tables (ignoring system tables)
            tables = [t.table_name for t in cursor.tables(tableType='TABLE')]
            for table in tables:
                query = f"SELECT * FROM [{table}]"
                all_tables_data[table] = pd.read_sql(query, conn)
    except Exception as e:
        print(f"Error reading Access file {filepath}: {e}")
    return all_tables_data

def consolidate():
    all_dfs = []

    # 1. Process Excel Files
    excel_files = glob.glob(os.path.join(SOURCE_FOLDER, "*.xlsx"))
    for file in excel_files:
        print(f"Processing Excel: {file}")
        # Read all sheets into a dictionary of DataFrames
        excel_data = pd.read_excel(file, sheet_name=None)
        for sheet_name, df in excel_data.items():
            df['Source_File'] = os.path.basename(file)
            df['Source_Sheet_Table'] = sheet_name
            all_dfs.append(df)

    # 2. Process MS Access Files
    access_files = glob.glob(os.path.join(SOURCE_FOLDER, "*.accdb"))
    for file in access_files:
        print(f"Processing Access: {file}")
        access_data = get_access_data(file)
        for table_name, df in access_data.items():
            df['Source_File'] = os.path.basename(file)
            df['Source_Sheet_Table'] = table_name
            all_dfs.append(df)

    # 3. Combine and Save
    if all_dfs:
        # ignore_index=True is used to create a fresh index for the merged data
        master_df = pd.concat(all_dfs, ignore_index=True, sort=False)
        master_df.to_excel(OUTPUT_FILE, index=False)
        print(f"Success! Consolidated data saved to {OUTPUT_FILE}")
    else:
        print("No compatible files found in the source folder.")

if __name__ == "__main__":
    consolidate()
